In [2]:
merged_f = open("../../afesm5/taxonomy/taxdump/merged.dmp")

cur_id2prev_id = {}

while True:
  line = merged_f.readline()

  if not line:
    break

  tokens = line.strip().split("\t|\t")
  cur_id = int(tokens[1].strip('\t|'))
  prev_id = int(tokens[0])

  if cur_id in cur_id2prev_id:
    cur_id2prev_id[cur_id].append(prev_id)
  else:
    cur_id2prev_id[cur_id] = [prev_id]

merged_f.close()

cur_id2prev_id

{74109: [12, 338050],
 29: [30],
 184914: [36],
 42: [37],
 39: [46],
 32033: [67],
 155892: [76, 80, 69394],
 74311: [77],
 74313: [79],
 34102: [90],
 34103: [91],
 2734912: [111,
  155397,
  155398,
  155399,
  155400,
  155403,
  155405,
  155406,
  328552],
 125: [121],
 99: [129, 61655, 204473],
 223389: [130],
 183: [177],
 210: [219],
 23: [229, 213786, 242623],
 24: [230],
 143223: [257],
 143224: [260, 29537],
 37931: [261, 388925, 1471255],
 274: [273],
 80866: [284, 95164, 1683560, 2447894],
 47421: [289],
 40137: [291],
 301: [330, 32041, 1437760, 1472039],
 50946: [334],
 456327: [341],
 40324: [345,
  49930,
  86190,
  169426,
  450641,
  450642,
  450643,
  1270204,
  1316735,
  1579528,
  1685430,
  1685431],
 29444: [348],
 359: [360, 270638],
 362: [363, 2756240],
 358: [364,
  365,
  366,
  367,
  368,
  369,
  33048,
  450636,
  450640,
  513651,
  515156,
  861208,
  1109803,
  1183401,
  1382282,
  2664455],
 186901: [377],
 382: [383, 34001, 300057, 1078746, 107

In [3]:
f = open("/fast/livinit/esmfold/databases/afesm_names.dmp")
taxName = {}

while True:
    line = f.readline()

    if not line:
        break

    tokens = line.split("\t|\t")
    if tokens[3] != "scientific name\t|\n":
      continue

    taxName[int(tokens[0])] = tokens[1]

f.close()

taxName

{1: 'root',
 2: 'Bacteria',
 6: 'Azorhizobium',
 7: 'Azorhizobium caulinodans',
 9: 'Buchnera aphidicola',
 10: 'Cellvibrio',
 11: 'Cellulomonas gilvus',
 13: 'Dictyoglomus',
 14: 'Dictyoglomus thermophilum',
 16: 'Methylophilus',
 17: 'Methylophilus methylotrophus',
 18: 'Pelobacter',
 19: 'Syntrophotalea carbinolica',
 20: 'Phenylobacterium',
 21: 'Phenylobacterium immobile',
 22: 'Shewanella',
 23: 'Shewanella colwelliana',
 24: 'Shewanella putrefaciens',
 25: 'Shewanella hanedai',
 27: 'halophilic eubacterium NRCC 41227',
 28: 'halophilic eubacterium',
 29: 'Myxococcales',
 31: 'Myxococcaceae',
 32: 'Myxococcus',
 33: 'Myxococcus fulvus',
 34: 'Myxococcus xanthus',
 35: 'Corallococcus macrosporus',
 38: 'Archangium disciforme',
 39: 'Archangiaceae',
 40: 'Stigmatella',
 41: 'Stigmatella aurantiaca',
 42: 'Cystobacter',
 43: 'Cystobacter fuscus',
 44: 'Melittangium',
 45: 'Melittangium lichenicola',
 47: 'Archangium',
 48: 'Archangium gephyra',
 49: 'Polyangiaceae',
 50: 'Chondromyc

In [4]:
f = open("/fast/livinit/esmfold/databases/afesm_nodes.dmp")

kid_parent = {}
taxRank = {}

while True:
    line = f.readline()

    if not line:
        break

    tokens = line.split("\t|\t")

    kid_parent[int(tokens[0])] = int(tokens[1])
    taxRank[int(tokens[0])] = tokens[2]

f.close()

In [18]:
dp = {}
dp_groupCode = {}

def dfs(cur):
    if cur == 131567:
        dp[cur] = cur
        dp_groupCode[cur] = 'cellular organism'
        return dp[cur]
    if not cur in kid_parent or cur == kid_parent[cur]:
        if cur == 1:
            dp[cur] = cur
            dp_groupCode[cur] = 'root'
        else:
            dp[cur] = False
            dp_groupCode[cur] = False
        return dp[cur]
    
    if taxRank[cur] == 'superkingdom':
        dp[cur] = cur
        dp_groupCode[cur] = 'superkingdom'
        return cur
    elif taxRank[cur] == 'family':
        dp[cur] = cur
        dp_groupCode[cur] = 'family and lower than family'
        return cur
    elif taxRank[cur] == 'species':
        dp[cur] = cur
        dp_groupCode[cur] = 'species and lower than species'
        return cur
    
    next = kid_parent[cur]

    if taxRank[next] == 'superkingdom':
        dp[cur] = cur
        dp_groupCode[cur] = 'lower than superkingdom'
        return cur

    dp[cur] = dfs(next)
    dp_groupCode[cur] = dp_groupCode[next]
    return dp[cur]

for key in kid_parent.keys():
    dfs(key)

In [19]:
f = open(f"../coreness/grouping_w_merged_dmp_gtdb-taxId_taxName_taxRank_groupTaxId_groupTaxName_groupTaxRank_groupName.tsv", "w")

N = 0
for key, val in dp.items():
    # N += 1
    # if N == 3:
    #     break
    if val:
        # print(key)
        # print(taxName[key])
        # print(dp_groupCode[key])
        # print(f"{key}\t{taxName[key]}\t{taxRank[key]}\t{val}\t{taxRank[val]}\t")
        # print(f"{key}\t{taxName[key]}\t{taxRank[key]}\t{val}\t{taxRank[val]}\t{taxName[val]}\t{taxRank[val]}\t{dp_groupCode[key]}\n")
        # N += 1
        # if N == 30:
        #     break
        f.write(f"{key}\t{taxName[key]}\t{taxRank[key]}\t{val}\t{taxRank[val]}\t{taxName[val]}\t{taxRank[val]}\t{dp_groupCode[key]}\n")
        if key in cur_id2prev_id:
            for prev_id in cur_id2prev_id[key]:
                # f.write(f"{prev_id}\t{val}\n")
                f.write(f"{prev_id}\t{taxName[key]}\t{taxRank[key]}\t{val}\t{taxRank[val]}\t{taxName[val]}\t{taxRank[val]}\t{dp_groupCode[key]}\n")

f.close()

In [20]:
f = open(f"../coreness/grouping_w_merged_dmp_gtdb-taxId_taxName_taxRank_groupName.tsv", "w")

N = 0
for key, val in dp.items():
    # N += 1
    # if N == 3:
    #     break
    if val:
        # print(key)
        # print(taxName[key])
        # print(dp_groupCode[key])
        # print(f"{key}\t{taxName[key]}\t{taxRank[key]}\t{val}\t{taxRank[val]}\t")
        # print(f"{key}\t{taxName[key]}\t{taxRank[key]}\t{val}\t{taxRank[val]}\t{taxName[val]}\t{taxRank[val]}\t{dp_groupCode[key]}\n")
        # N += 1
        # if N == 30:
        #     break
        f.write(f"{key}\t{taxName[key]}\t{taxRank[key]}\t{dp_groupCode[key]}\n")
        if key in cur_id2prev_id:
            for prev_id in cur_id2prev_id[key]:
                # f.write(f"{prev_id}\t{val}\n")
                f.write(f"{prev_id}\t{taxName[key]}\t{taxRank[key]}\t{dp_groupCode[key]}\n")

f.close()